In [1]:
import os
from pathlib import Path
os.chdir('../')
%pwd

'c:\\Users\\Saumil\\Git\\text_summarizer'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_dir

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_dir([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy =  params.evaluation_strategy,
            eval_steps =  params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps,
            root_dir= config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt
        )

        return model_trainer_config

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset, load_from_disk
import torch

In [12]:
class ModelTrainer:
    def __init__(
        self,
        config: ModelTrainerConfig,
    ):
        self.config = config
    
    def train(self):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        training_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model_pegasus,
            args=training_args,
            train_dataset=dataset_samsum_pt['test'],
            eval_dataset=dataset_samsum_pt['validation']
        )

        trainer.train()
        
        model_pegasus.save_pretrained(Path(f'{self.config.root_dir}/pegasus-samsum-model'))
        tokenizer.save_pretrained(Path(f'{self.config.root_dir}/pegasus-samsum-tokenizer'))

In [13]:
try:
    config = ConfigurationManager()
    model_training = ModelTrainer(config.get_model_trainer_config())
    model_training.train()
    '''logger.info('configuration is created')'''
except Exception as e:
    raise e

[2024-01-21 21:37:09,767: INFO: common: Read config\config.yaml successfully]
[2024-01-21 21:37:09,783: INFO: common: Read params.yaml successfully]
[2024-01-21 21:37:09,786: INFO: common: artifacts/model_trainer already exists]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: '>' not supported between instances of 'str' and 'int'